<a href="https://colab.research.google.com/github/vinicius-vargas/robust-market-screener/blob/main/stock_valuation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q pandas-datareader

In [ ]:
!pip install -q yfinance

In [ ]:
! pip install -q investpy # Investing.com

In [ ]:
!pip install -q fundamentus

In [ ]:
### Setting up libraries
from google.colab import drive
import numpy as np
import pandas as pd
from datetime import datetime
import time
from pandas_datareader import data as pdr
import yfinance as yf
import investpy as inv
import fundamentus as fd
import warnings

# Connect to Google Drive (My data lake)
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
### Read data from my data lake
df=pd.read_csv('/content/drive/My Drive/data_lake/alpha_raking.csv')

# Get current date t
curr_date = datetime.today().strftime('%Y-%m-%d')

# Check if the Last Update is equal today
df = df[(df.last_update == curr_date)]

df.head(3)

,ticker,qtd_dias,vix,fed_3_y,s_p,msci_top_ex_us,msci_emg,bond_emg,cmmdt,usd_real,mkt,size,qld,momnt,liq,return,vol,alpha,r_score,last_update
0,SQIA3.SA,2528,0.013,-0.001,0.305,-0.222,0.083,0.243,-0.001,-0.095,0.681,0.828,-0.550,0.135,0.118,1.879,0.02801,0.00110,0.238,2022-11-17
1,RADL3.SA,4400,-0.032,-0.000,-0.061,-0.154,0.071,0.058,-0.046,-0.022,0.649,0.041,-0.449,-0.012,0.047,6.218,0.02055,0.00100,0.191,2022-11-17
2,UNIP6.SA,4400,0.004,0.002,0.214,-0.029,0.010,0.073,0.029,-0.004,0.680,0.177,-0.093,0.159,0.272,3.371,0.02627,0.00097,0.171,2022-11-17


In [ ]:
# function to check if a value is '-'
def compare(x):
  if x=='-': 
    return 0.0
  else:
    return float(x) / 100


# Select tickers to get fundamentalist informations #'DEXP3.SA'
lista = df['ticker']

# Create dataframe to save fundamental indexes
data = pd.DataFrame()

for ticker in lista:
  ### Get the Historical Company Performance - Gross and Net Margin
  #################################################################
  print(ticker)

  # Annual - to get the last 5 years
  asst_data = yf.Ticker(ticker)

  df_fd = fd.get_papel(ticker[:-3]) 

  # Get total of shares
  try:
    asst_data.shares.sort_index(ascending=False)
  
  # Deal with the absence of share information
  except:
    total_shares = pd.DataFrame(
        {
            'BasicShares': [df_fd['Nro_Acoes'].astype(int)[0]] * 4
         },
         index = asst_data.balance_sheet.T.index.year
    ).sort_index(ascending=False)
    
  
  # In this case, return total shares by year
  else:
    total_shares = asst_data.shares.sort_index(ascending=False)

  
  # Function do Get the Balance Sheet (or Balanco Patrimonial) 
  asst_annual_bs = asst_data.balance_sheet.T
  asst_annual_bs.index = asst_annual_bs.index.year
  asst_annual_bs = pd.concat([asst_annual_bs, total_shares], axis=1)

  # Function do Get the Income Statement (or DRE)
  asst_annual_dre = asst_data.financials.T
  asst_annual_dre.index = asst_annual_dre.index.year
  asst_annual_dre = pd.concat([asst_annual_dre, total_shares], axis=1)

  ### Get Gross Margin and Net Margin from DRE
  asst_annual_dre['ticker'] = ticker

  asst_annual_dre['Gross Margin'] = asst_annual_dre['Gross Profit'] / asst_annual_dre['Total Revenue']
  asst_annual_dre['Net Margin'] = asst_annual_dre['Net Income'] / asst_annual_dre['Total Revenue']

  # Agregate number by ticker - And uses Median (Because it is robust)
  hist_perf = asst_annual_dre.groupby('ticker').agg(
      {
      'Gross Margin': np.median,
      'Net Margin': np.median
      }
  )



  ### Get the Current Company Performance - Gross and Net Margin
  ##############################################################

  # Function do Get the Income Statement (or DRE)
  asst_qr_dre = asst_data.quarterly_financials.T

  ### Get Gross Margin and Net Margin from DRE
  asst_qr_dre['ticker'] = ticker

  asst_qr_dre['Gross Margin'] = asst_qr_dre['Gross Profit'].sum() / asst_qr_dre['Total Revenue'].sum()
  asst_qr_dre['Net Margin'] = asst_qr_dre['Net Income'].sum() / asst_qr_dre['Total Revenue'].sum()

  currt_perf = asst_qr_dre.groupby('ticker').agg(
      {
      'Gross Margin': max,
      'Net Margin': max
      }
  )

  ### Get Current Current Ratio
  #############################

  curr_ratio = compare(df_fd['Liquidez_Corr'][0])



  ### Get the Historical Dividend Yield
  #####################################

  # Get Historical Prices & Dividends
  asst_data = asst_data.history(period = '10y')

  asst_data['Year'] = asst_data.index.year

  ### Get the last price of each year
  last_prices = asst_data.groupby('Year')['Close'].agg(['last'])

  ### Calculate the Dividend payed by year
  sun_div = asst_data[asst_data.Dividends != 0].groupby('Year')['Dividends'].agg(['sum'])

  ### Grouping Last Price with Dividends Sum
  asst_div_data = pd.concat([last_prices, sun_div], axis=1)

  asst_div_data['yield'] = asst_div_data['sum'] / asst_div_data['last']

  hist_div = round(asst_div_data['yield'].median() * 100, 2)



  ### Get Asset Historical Multiples for Valuation
  ################################################

  # Get RLA 
  RLA = asst_annual_dre['Total Revenue'] / asst_annual_dre['BasicShares']

  # Get VPA
  VPA = asst_annual_bs['Total Stockholder Equity'] / asst_annual_dre['BasicShares']

  # Get LPA 
  LPA = asst_annual_dre['Net Income'] / asst_annual_dre['BasicShares']

  # Concate datasets & Rename Columns - To make it more simple
  hist_index = pd.concat([RLA, VPA, LPA, last_prices[last_prices.index.isin(LPA.index)]], axis=1)

  hist_index.columns = ['RLA', 'VPA', 'LPA', 'Close']

  # Get Fundamental Indicators
  hist_index = hist_index.assign(
      PSA = hist_index['Close'] / hist_index['RLA'],
      PVP = round(hist_index['Close'] / hist_index['VPA'], 2),
      PL = hist_index['Close'] / hist_index['LPA']
  ).drop(['RLA', 'VPA', 'LPA', 'Close'], axis=1).median()



  ### Get Current Multiples  
  curr_index = pd.DataFrame(
      {
       'PSR':[compare(df_fd['PSR'][0])],
       'PVP':[compare(df_fd['PVP'][0])],
       'PL':[compare(df_fd['PL'][0])],
       }
  )


  ### Final Dataset - Fundamentalist Performance & Index
  ######################################################

  final_data_fund = pd.DataFrame(
      {
      'ticker': ticker,
      'Setor': df_fd['Setor'][0],
      'Mg. Bruta Med.': [round(hist_perf['Gross Margin'][0] * 100, 2)],
      'Mg. Bruta Hj': [round(currt_perf['Gross Margin'][0] * 100, 2)],
      'Mg. Net Med': [round(hist_perf['Net Margin'][0] * 100, 2)],
      'Mg. Net Hj':[round(currt_perf['Net Margin'][0] * 100, 2)],
      'Liq Corrente': [curr_ratio],
      'Div. Yield Med': [hist_div],
      'Div. Yield Hj': [float(df_fd['Div_Yield'][0][:-1])],
      'PSR Med': [round(hist_index[0], 2)],
      'PSR Hj':[curr_index['PSR'][0]],
      'PVP Med':[round(hist_index[1], 2)],
      'PVP Hj':[curr_index['PVP'][0]],
      'PL Med':[round(hist_index[2], 2)],
      'PL Hj':[curr_index['PL'][0]] 
      }
  )
  
  time.sleep(1)
  
  data = data.append(final_data_fund, ignore_index=True)

In [ ]:
data

,ticker,Setor,Mg. Bruta Med.,Mg. Bruta Hj,Mg. Net Med,Mg. Net Hj,Liq Corrente,Div. Yield Med,Div. Yield Hj,PSR Med,PSR Hj,PVP Med,PVP Hj,PL Med,PL Hj
0,SQIA3.SA,Programas e Serviços,35.65,40.94,2.14,4.63,1.00,0.59,0.4,5.52,2.59,3.22,1.83,84.15,44.51
1,RADL3.SA,Comércio e Distribuição,29.73,30.40,3.26,3.18,1.52,0.90,1.1,1.83,1.40,8.70,7.69,49.66,44.11
2,UNIP6.SA,Químicos,35.41,46.83,12.36,31.70,2.06,11.31,14.7,0.86,1.46,1.91,4.05,7.18,6.14
3,STBP3.SA,Transporte,26.27,43.91,0.95,22.04,1.99,4.38,12.0,3.47,3.41,2.09,3.03,165.88,15.46
4,MOAR3.SA,Holdings Diversificadas,42.06,-141.30,4994.13,109852.17,2.94,10.19,4.0,966.37,129168.00,2.00,7.18,12.48,56.26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,TEND3.SA,Construção Civil,32.18,10.95,10.34,-18.14,1.34,1.66,0.0,1.16,0.22,1.76,0.59,9.82,-0.75
80,NEOE3.SA,Energia Elétrica,17.49,24.91,8.31,10.23,1.30,1.71,5.3,0.65,0.42,0.98,0.68,7.39,4.11
81,RNEW4.SA,Energia Elétrica,27.94,12.81,-44.41,127.13,0.69,NaN,0.0,7.53,3.27,-0.91,3.51,8.67,3.02
82,KLBN11.SA,Madeira e Papel,26.63,25.74,10.64,0.63,2.35,3.28,6.4,12.50,1.33,11.84,2.89,61.09,5.30


In [ ]:
######################################
### Filter All TOP Stocks - And Cheap!
######################################

final_data = data[
    (data['Mg. Bruta Med.'] >= 15)
    & (data['Mg. Bruta Hj'] >= 15)

    & (data['Mg. Net Med'] > 5)
    & (data['Mg. Net Hj'] > 5)

    & (data['Liq Corrente'] >= 1.1)

    & (data['Div. Yield Med'] >= 4.25)
    & (data['Div. Yield Hj'] >= 5.5)
    & (data['Div. Yield Hj'] < 20)

    & (data['PSR Med'] <= 10)
    & ((data['PSR Hj'] <= 4.5) | (data['PSR Hj'] <= data['PSR Med']))

    & (data['PVP Med'] <= 3.5)
    & ((data['PVP Hj'] <= 4) | (data['PVP Hj'] <= data['PVP Med']))


    & (data['PL Med'] > 0)
    & (data['PL Med'] <= 20)
    & (data['PL Hj'] > 0)
    & ((data['PL Hj'] <= 7.5) | (data['PL Hj'] <= data['PL Med']))

  ].reset_index(drop=True)

final_data

,ticker,Setor,Mg. Bruta Med.,Mg. Bruta Hj,Mg. Net Med,Mg. Net Hj,Liq Corrente,Div. Yield Med,Div. Yield Hj,PSR Med,PSR Hj,PVP Med,PVP Hj,PL Med,PL Hj
0,TAEE11.SA,Energia Elétrica,88.97,81.71,60.34,62.57,3.71,13.66,14.3,7.17,4.53,2.40,1.84,11.69,6.91
1,FESA4.SA,Siderurgia e Metalurgia,28.93,48.83,19.82,36.55,3.94,5.23,7.2,1.22,1.50,0.92,1.53,7.08,4.07
2,GRND3.SA,"Tecidos, Vestuário e Calçados",45.85,41.66,25.37,25.23,7.35,7.51,5.7,3.47,2.40,1.85,1.42,12.37,10.20
3,CGRA4.SA,Comércio,53.40,54.05,21.36,24.91,2.51,5.23,11.8,1.13,0.85,0.92,0.71,5.78,4.95
4,AGRO3.SA,Agropecuária,50.73,43.28,36.00,67.06,2.51,5.70,19.7,3.77,1.61,1.65,1.20,9.48,6.26
5,ENBR3.SA,Energia Elétrica,16.14,18.05,9.63,11.64,1.48,4.49,10.4,0.66,0.68,1.04,1.04,6.29,5.52


In [ ]:
####################################################
### Filter All TOP Stocks - But Expensive - TO WATCH
####################################################

to_watch = data[
    (data['Mg. Bruta Med.'] >= 25)
    & (data['Mg. Bruta Hj'] >= 25)

    & (data['Mg. Net Med'] > 5)
    & (data['Mg. Net Hj'] > 5)

    & (data['Liq Corrente'] >= 1)

    & (data['Div. Yield Med'] >= 4.5)
    & (data['Div. Yield Hj'] < 20)

  ].reset_index(drop=True)

to_watch = to_watch[~to_watch.ticker.isin(final_data.ticker)]

to_watch

,ticker,Setor,Mg. Bruta Med.,Mg. Bruta Hj,Mg. Net Med,Mg. Net Hj,Liq Corrente,Div. Yield Med,Div. Yield Hj,PSR Med,PSR Hj,PVP Med,PVP Hj,PL Med,PL Hj
0,UNIP6.SA,Químicos,35.41,46.83,12.36,31.70,2.06,11.31,14.7,0.86,1.46,1.91,4.05,7.18,6.14
3,EMAE4.SA,Energia Elétrica,26.20,33.53,30.21,22.13,5.49,6.05,3.8,4.00,2.63,2.33,1.51,12.67,13.48
5,DIRR3.SA,Construção Civil,34.25,35.71,7.21,8.04,3.92,7.23,3.6,1.16,0.93,1.50,1.45,14.24,11.18
8,GUAR3.SA,Comércio,58.73,57.00,6.93,5.59,1.51,4.77,7.4,1.20,0.45,1.63,0.72,9.10,14.66
9,EGIE3.SA,Energia Elétrica,44.56,52.09,23.19,14.37,1.26,7.36,5.6,2.58,2.80,4.05,4.02,13.52,17.48
10,FLRY3.SA,Serv.Méd.Hospit. Análises e Diagnósticos,29.72,28.40,9.89,8.16,1.38,5.47,4.7,2.43,1.25,3.94,2.85,22.95,15.67
11,TRPL4.SA,Energia Elétrica,72.95,69.24,56.82,47.45,5.29,14.76,0.8,3.24,2.67,1.06,0.94,5.01,5.86
12,CSMG3.SA,Água e Saneamento,39.50,32.78,13.42,7.10,1.00,6.91,2.6,1.18,0.92,0.93,0.78,8.97,8.81
13,VALE3.SA,Mineração,51.51,52.75,16.02,41.18,1.07,4.76,9.2,1.47,1.68,1.62,2.19,5.39,3.69
14,QUAL3.SA,Serv.Méd.Hospit. Análises e Diagnósticos,80.98,80.04,18.98,13.89,1.88,6.53,3.9,3.41,0.92,4.64,1.28,18.27,8.28


In [ ]:
###############################################
### Filter Only Cheap Stocks - But not that BAD
###############################################

cheap_to_watch = data[
    (data['Mg. Bruta Med.'] >= 15)
    & (data['Mg. Bruta Hj'] >= 15)

    & (data['Mg. Net Med'] > 0)
    & (data['Mg. Net Hj'] > 0)

    & (data['Liq Corrente'] >= 0.95)

    & (data['Div. Yield Med'] >= 3)
    & (data['Div. Yield Hj'] >= 3)
    & (data['Div. Yield Hj'] < 20)

    & (data['PSR Med'] <= 10)
    & ((data['PSR Hj'] <= 1.5) | (data['PSR Hj'] <= data['PSR Med']))

    & (data['PVP Med'] <= 3.5)
    & ((data['PVP Hj'] <= 2) | (data['PVP Hj'] <= data['PVP Med']))


    & (data['PL Med'] > 0)
    & (data['PL Med'] <= 20)
    & (data['PL Hj'] > 0)
    & ((data['PL Hj'] <= 7) | (data['PL Hj'] <= data['PL Med']))
  
  ].reset_index(drop=True)

cheap_to_watch = (
    cheap_to_watch[
        (~cheap_to_watch.ticker.isin(final_data.ticker))
        & (~cheap_to_watch.ticker.isin(to_watch.ticker))
    ]
    .reset_index(drop=True)
)

cheap_to_watch

,ticker,Setor,Mg. Bruta Med.,Mg. Bruta Hj,Mg. Net Med,Mg. Net Hj,Liq Corrente,Div. Yield Med,Div. Yield Hj,PSR Med,PSR Hj,PVP Med,PVP Hj,PL Med,PL Hj
0,SHUL4.SA,Máquinas e Equipamentos,22.60,25.25,10.23,12.53,2.84,7.94,3.8,0.82,0.81,1.42,1.55,7.91,6.47
1,ROMI3.SA,Máquinas e Equipamentos,29.23,31.31,15.83,12.19,1.86,3.03,5.1,1.03,0.79,1.22,1.16,6.33,6.48
2,ENAT3.SA,"Petróleo, Gás e Biocombustíveis",32.32,39.96,34.94,38.35,1.69,4.20,12.0,2.69,1.71,0.99,0.95,12.06,4.24


In [ ]:
###########################################
### FINAL STOCK PICKING - IT MUST BE MANUAL 
###########################################

final_data = (
    final_data
    .append(to_watch[to_watch.ticker == 'UNIP6.SA'])
    .append(to_watch[to_watch.ticker == 'EGIE3.SA'])
    .append(to_watch[to_watch.ticker == 'TRPL4.SA'])
    #.append(cheap_to_watch)
    .drop_duplicates(subset='ticker', keep="first")
    .reset_index(drop=True)
)

#final_data.ticker[4] = 'TAEE11.SA'

final_data

,ticker,Setor,Mg. Bruta Med.,Mg. Bruta Hj,Mg. Net Med,Mg. Net Hj,Liq Corrente,Div. Yield Med,Div. Yield Hj,PSR Med,PSR Hj,PVP Med,PVP Hj,PL Med,PL Hj
0,TAEE11.SA,Energia Elétrica,88.97,81.71,60.34,62.57,3.71,13.66,14.3,7.17,4.53,2.40,1.84,11.69,6.91
1,FESA4.SA,Siderurgia e Metalurgia,28.93,48.83,19.82,36.55,3.94,5.23,7.2,1.22,1.50,0.92,1.53,7.08,4.07
2,GRND3.SA,"Tecidos, Vestuário e Calçados",45.85,41.66,25.37,25.23,7.35,7.51,5.7,3.47,2.40,1.85,1.42,12.37,10.20
3,CGRA4.SA,Comércio,53.40,54.05,21.36,24.91,2.51,5.23,11.8,1.13,0.85,0.92,0.71,5.78,4.95
4,AGRO3.SA,Agropecuária,50.73,43.28,36.00,67.06,2.51,5.70,19.7,3.77,1.61,1.65,1.20,9.48,6.26
5,ENBR3.SA,Energia Elétrica,16.14,18.05,9.63,11.64,1.48,4.49,10.4,0.66,0.68,1.04,1.04,6.29,5.52
6,UNIP6.SA,Químicos,35.41,46.83,12.36,31.70,2.06,11.31,14.7,0.86,1.46,1.91,4.05,7.18,6.14
7,EGIE3.SA,Energia Elétrica,44.56,52.09,23.19,14.37,1.26,7.36,5.6,2.58,2.80,4.05,4.02,13.52,17.48
8,TRPL4.SA,Energia Elétrica,72.95,69.24,56.82,47.45,5.29,14.76,0.8,3.24,2.67,1.06,0.94,5.01,5.86


In [ ]:
### Save the output inside Google Drive
final_data.to_csv('/content/drive/My Drive/data_lake/stock_valuation.csv', encoding='utf-8', index=False)